In [1]:
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from ast import literal_eval
from sklearn.model_selection import train_test_split

import os
import random
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)

In [3]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

df["choices_len"] = df["choices"].apply(len)
df['choices_len'].value_counts(dropna=False)

df4 = df[df['choices_len'] == 4]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [4]:
train_df, valid_df = train_test_split(
    df4,
    test_size=0.15,        
    random_state=42,
)

train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_ds = Dataset.from_pandas(valid_df.reset_index(drop=True))

dataset = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 673
    })
    validation: Dataset({
        features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len'],
        num_rows: 119
    })
})

In [5]:
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )

### 정책별 System Prompt 함수
def get_system_message(row, system_prompts, prompt_policy):
    """
    row: 하나의 데이터 행
    system_prompts: {choices_len: {version: prompt_text}}
    prompt_policy: {choices_len: version}
    """
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    return system_prompts[choices_len][version]

SYSTEM_PROMPT_4_V1 = (
    "당신은 **지식 추론(Knowledge Inference) 전문가**입니다. "
    "이 유형은 정답이 지문에 그대로 쓰여 있지 않을 수 있으며, 지문은 '조건/단서'를 제공합니다. "
    "지문에서 주어진 조건을 정확히 반영하고, 그 조건과 모순되지 않는 범위에서 일반적으로 알려진 지식을 적용해 "
    "가장 타당한 선택지 하나를 고르십시오."
)

SYSTEM_PROMPT_5_V1 = (
    "당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다. "
    "이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다. "
    "당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.\n\n"
)

# PROMPT_POLICY:
# - 이번 실험(run)에서 "어떤 system prompt 버전을 사용할지"를 결정하는 설정값
# - choices_len(4 or 5) → 사용할 prompt 버전(v1, v2, ...)
# - 실험을 바꿀 때는 이 딕셔너리만 수정

SYSTEM_PROMPTS = {
    4: {
        "v1": SYSTEM_PROMPT_4_V1,
    },
    5: {
        "v1": SYSTEM_PROMPT_5_V1,
    }
}

SYSTEM_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [69]:
### 정책별 User Prompt 함수
def get_user_message(row, user_prompts, prompt_policy):
    """
    row: 데이터 행
    user_prompts: 템플릿 저장소
    prompt_policy: 버전 정책
    """
    # 메타 데이터 확인
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    
    # 해당 버전의 템플릿 세트 가져오기 (plus, no_plus가 들어있음)
    template_set = user_prompts[choices_len][version]
    
    # 데이터 준비
    paragraph = row['paragraph']
    question = row['question']
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(row['choices'])])
    q_plus = row.get('question_plus', None)
    
    # 분기 처리 및 포맷팅 (여기가 핵심!)
    # q_plus가 존재하고, nan이 아닐 때 -> Plus 템플릿 사용
    if q_plus and str(q_plus) != 'nan':
        return template_set["plus"].format(
            paragraph=paragraph,
            question_plus=q_plus, # 여기 들어감
            question=question,
            choices=choices_str
        )
    # q_plus가 없을 때 -> No Plus 템플릿 사용
    else:
        return template_set["no_plus"].format(
            paragraph=paragraph,
            question=question,
            choices=choices_str
        )
    
# =========================
# User Prompt Templates (V1)
# =========================

# 4지선다 + <보기> 있음
USER_PROMPT_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 4지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_4_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문이 주는 조건/단서를 먼저 정리하세요. (무엇을 가정/설명하고 있는지)
2. 필요하면 일반적으로 알려진 지식(개념/원리/사례)을 적용하되, 지문 조건과 모순되면 안 됩니다.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 4지선다 + <보기> 있음 + RAG 힌트 추가
USER_PROMPT_PLUS_4_RAG_V1 = """### 참고 정보 (검색된 지식)
{retrieved_context}

### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 상단의 '참고 정보'와 '지문'을 바탕으로 문제 풀이에 필요한 핵심 단서를 정리하세요.
2. '참고 정보'의 내용이 지문과 상호 보완적일 경우 이를 적극적으로 활용하여 추론하세요.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

# 4지선다 + <보기> 없음 + RAG 힌트 추가
USER_PROMPT_NO_PLUS_4_RAG_V1 = """### 참고 정보 (검색된 지식)
{retrieved_context}

### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 상단의 '참고 정보'와 '지문'을 바탕으로 문제 풀이에 필요한 핵심 단서를 정리하세요.
2. 지문 내용만으로 부족할 경우 '참고 정보'에 기술된 개념이나 사실을 근거로 판단하세요.
3. 선택지 중 조건을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

# 4지선다 + <보기> 없음 + RAG 힌트 추가 (V2: 지문 우선순위 강화)
USER_PROMPT_NO_PLUS_4_RAG_V2 = """### 참고 정보 (검색된 지식)
{retrieved_context}

### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 상단의 '참고 정보'와 '지문'을 대조하여 문제 풀이에 필요한 핵심 팩트를 정리하세요.
2. 지문 내용이 부족할 경우 '참고 정보'를 활용하되, **반드시 '지문'에 명시된 조건과 모순되지 않는지 확인하세요.** (지문 조건이 최우선입니다.)
3. 선택지 중 지문의 상황을 가장 잘 만족하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

USER_PROMPT_PLUS_4_RAG_V2 = """### 참고 정보 (검색된 지식)
{retrieved_context}

### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. '지문'의 조건과 '참고 정보'의 핵심 단서를 각각 정리하세요.
2. '참고 정보'를 활용하되, 반드시 '지문'에 명시된 조건과 모순되지 않는지 먼저 확인하세요. (지문 조건이 최우선입니다.)
3. 선택지 중 지문의 상황을 가장 잘 만족하고, 참고 정보와 일치하는 것 하나만 고르세요.

정답은 1~4 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

# 5지선다 + <보기> 있음
USER_PROMPT_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 보기
{question_plus}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""


# 5지선다 + <보기> 없음
USER_PROMPT_NO_PLUS_5_V1 = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. 지문을 끝까지 읽고 핵심 정보를 정리하세요.
2. 질문이 요구하는 정보(수치/인물/원인/결과/요지 등)가 무엇인지 정확히 확인하세요.
3. 각 선택지가 지문의 어느 부분과 일치하는지 1:1로 대조하세요.
4. 지문과 모순되거나 지문에 근거가 없는 선택지는 제외하세요.
5. 가장 확실한 근거를 가진 선택지 번호 하나만 선택하세요.

정답은 1~5 중 하나의 정수로만 출력하세요. 다른 글자는 출력하지 마세요.
정답:"""

USER_PROMPTS = {
    4: {
        "v1": {
            "plus": USER_PROMPT_PLUS_4_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_V1,
        },
        # "v2": {...}
    },
    5: {
        "v1": {
            "plus": USER_PROMPT_PLUS_5_V1,
            "no_plus": USER_PROMPT_NO_PLUS_5_V1,
        },
        # "v2": {...}
    }
}

USER_PROMPT_POLICY = {
    4: "v1",
    5: "v1",
}

In [7]:
# 공통 Assistant Prompt 함수
def get_assistant_message(row):
    """
    Assistant 메시지 생성 함수.
    Qwen3 모델의 토크나이저 템플릿이 자동으로 <think> 태그를 처리하므로,
    여기서는 순수한 정답(Label) 텍스트만 반환
    """
    return str(row['answer'])

In [9]:
def build_messages(example):
    """
    원본 example(row)로부터 학습용 chat messages를 구성한다.
    - choices_len(4/5) 및 question_plus 유무에 따라 system/user 프롬프트를 선택
    - assistant는 정답 숫자만
    - 이후 평가/추적용으로 id, label도 함께 유지
    """
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)
    asst_msg = get_assistant_message(example)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": asst_msg},
        ],
        "label": int(example["answer"]),
    }


def build_test_messages(example):
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
        ]
    }

def to_text(example):
    """
    messages(list[dict])를 tokenizer의 chat_template 규칙에 따라
    단일 텍스트로 직렬화한다.
    """
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,  
    )
    return {"text": text}

def to_test_text(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=True, 
    )
    return {"text": text}

def tokenize_fn(example, truncation=True, max_length=2048, padding=False):
    """
    batched=True면 example["text"]는 List[str]
    batched=False면 example["text"]는 str
    """
    tok_kwargs = dict(truncation=truncation, padding=padding)
    if truncation is True:
        tok_kwargs["max_length"] = max_length

    out = tokenizer(example["text"], **tok_kwargs)
    
    return {
        "input_ids": out["input_ids"],
        "attention_mask": out["attention_mask"],
    }


In [ ]:
orig_cols = dataset["train"].column_names
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    fn_kwargs={"truncation": True, "max_length": 2048, "padding": False},
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    keep_in_memory=True,
    desc="Tokenizing",
)

In [11]:
orig_cols = dataset["train"].column_names
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    fn_kwargs={"truncation": True, "max_length": 2048, "padding": False},
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    keep_in_memory=True,
    desc="Tokenizing",
)

Build messages:   0%|          | 0/673 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/119 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/673 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/673 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/119 [00:00<?, ? examples/s]

In [12]:
print("\n=== 변환 완료 ===")
print("Train 개수:", len(tokenized_dataset["train"]))
print("첫 번째 샘플 Keys:", tokenized_dataset["train"][0].keys())


=== 변환 완료 ===
Train 개수: 673
첫 번째 샘플 Keys: dict_keys(['id', 'label', 'input_ids', 'attention_mask'])


In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 673
    })
    validation: Dataset({
        features: ['id', 'label', 'input_ids', 'attention_mask'],
        num_rows: 119
    })
})

In [14]:
### 검증용
test_ds_msg = valid_ds.map(
    build_test_messages,
    batched=False,
    desc="Build messages",
)
test_ds_text = test_ds_msg.map(
    to_test_text,
    batched=False,
    desc="Serialize to text",
)

Build messages:   0%|          | 0/119 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/119 [00:00<?, ? examples/s]

In [17]:
response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

In [18]:
DIGIT_IDS = [16, 17, 18, 19, 20]  # '1'~'5'

def preprocess_logits_for_metrics(logits, labels, pos_from_tail=4):
    """
    반환: (batch, 5)  -> '1'~'5'에 해당하는 logits만 뽑아서 metrics 단계로 전달
    """
    # Trainer가 (logits, ...) 튜플을 줄 때가 있어서 정리
    if isinstance(logits, tuple):
        logits = logits[0]  # (B, L, V)

    # labels: (B, L), pad/무시 영역은 -100일 가능성이 큼
    # real_len = 마지막으로 labels != -100 인 위치 + 1 로 복원
    labels_t = torch.as_tensor(labels)
    not_ignored = (labels_t != -100)

    # 샘플별로 마지막 not_ignored 위치 찾기
    # (뒤에서부터 True 찾기)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)          # (B,)
    has_any = not_ignored.any(dim=1)                             # (B,)
    # real_len = seq_len - last_true_from_end
    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end

    # 만약 labels가 전부 -100인 샘플이 있으면(비정상) 그냥 seq_len로 처리
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos = (real_len - pos_from_tail).clamp(min=0, max=seq_len-1) # (B,)

    # (B, V)로 해당 위치의 logits만 gather
    logits_t = torch.as_tensor(logits)                           # (B, L, V)
    batch_idx = torch.arange(logits_t.size(0), device=logits_t.device)
    picked = logits_t[batch_idx, pos, :]                         # (B, V)

    # digit ids만 슬라이스 -> (B, 5)
    picked_digits = picked[:, DIGIT_IDS]
    return picked_digits

def compute_metrics(eval_pred, label_pos_from_tail=3):
    """
    eval_pred:
      - (predictions, label_ids) 튜플 형태가 가장 흔함
      - predictions: preprocess_logits_for_metrics가 반환한 (B, 5)
      - label_ids: (B, L) with -100 ignored
    반환: {"accuracy": ..., "macro_f1": ...}
    """
    if hasattr(eval_pred, "predictions"):
        preds, labels = eval_pred.predictions, eval_pred.label_ids
    else:
        preds, labels = eval_pred

    preds_t = torch.as_tensor(preds)
    pred_cls = torch.argmax(preds_t, dim=-1).cpu().numpy().astype(np.int64)  # (B,)

    labels_t = torch.as_tensor(labels)

    not_ignored = (labels_t != -100)
    rev = torch.flip(not_ignored, dims=[1])
    last_true_from_end = torch.argmax(rev.int(), dim=1)
    has_any = not_ignored.any(dim=1)

    seq_len = labels_t.size(1)
    real_len = seq_len - last_true_from_end
    real_len = torch.where(has_any, real_len, torch.full_like(real_len, seq_len))

    pos_label = (real_len - label_pos_from_tail).clamp(min=0, max=seq_len - 1)
    batch_idx = torch.arange(labels_t.size(0), device=labels_t.device)
    gold_tok = labels_t[batch_idx, pos_label].cpu().numpy().astype(np.int64) 

    gold_cls = gold_tok - DIGIT_IDS[0]  

    valid = (gold_cls >= 0) & (gold_cls < 5)
    pred_cls = pred_cls[valid]
    gold_cls = gold_cls[valid]

    acc = (pred_cls == gold_cls).mean() if len(gold_cls) > 0 else 0.0

    f1s = []
    for c in range(5):
        tp = np.sum((pred_cls == c) & (gold_cls == c))
        fp = np.sum((pred_cls == c) & (gold_cls != c))
        fn = np.sum((pred_cls != c) & (gold_cls == c))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
        f1s.append(f1)

    macro_f1 = float(np.mean(f1s)) if len(f1s) > 0 else 0.0

    return {"accuracy": float(acc), "macro_f1": macro_f1}

In [19]:
train_dataset = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["id", "label"])

In [20]:
# 1. 모델 이름 및 양자화 설정
MODEL_NAME = "Qwen/Qwen3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

# 2. 토크나이저 먼저 로드 및 PAD 설정 (중요!)
# 모델 로드 전에 PAD 토큰을 확정지어야 나중에 모델 설정에 바로 반영됩니다.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 3. 모델 로드 (양자화 및 장치 할당)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

# 4. 모델 설정 업데이트
model.config.use_cache = False
model.gradient_checkpointing_enable()

# 토크나이저의 PAD 설정을 모델에 동기화
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# 확인 출력
print(f"Final Pad Token: {tokenizer.pad_token} ({tokenizer.pad_token_id})")
print(f"Use Cache: {model.config.use_cache}, Grad Ckpt: {model.is_gradient_checkpointing}")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Final Pad Token: <|endoftext|> (151643)
Use Cache: False, Grad Ckpt: True


In [22]:
model = prepare_model_for_kbit_training(model)

# Attention proj만
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Attention + MLP까지 (성능 더 노리되 trainable 조금 증가)
# "gate_proj", "up_proj", "down_proj" -> FFN
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# target_modules = ["q_proj", "k_proj"]

target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [23]:
training_args = SFTConfig(
    output_dir="../../qwen-sft-results",
    
    num_train_epochs=2,
    max_seq_length=2048,
    packing=False,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    fp16=True,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # 전략 설정
    eval_strategy="steps",       # 주석 해제 (활성화)
    eval_steps=20,
    save_strategy="no",          # 저장 안 함 (비교용)
    load_best_model_at_end=False, # 저장 안 할 때는 False가 안전
    
    # 지표 및 로깅
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_steps=10,
    report_to="none",
)

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args=training_args,
)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,Accuracy,Macro F1
20,1.182100,0.132695,0.647059,0.507284
40,0.111800,0.110179,0.722689,0.571552
60,0.134900,0.107921,0.739496,0.574373
80,0.142500,0.112901,0.722689,0.573093
100,0.088500,0.109935,0.714286,0.561669
120,0.071700,0.108128,0.722689,0.567527
140,0.090800,0.104781,0.705882,0.554779
160,0.097600,0.104107,0.705882,0.554779


TrainOutput(global_step=170, training_loss=0.374595701343873, metrics={'train_runtime': 1405.3218, 'train_samples_per_second': 0.958, 'train_steps_per_second': 0.121, 'total_flos': 4.16753022432768e+16, 'train_loss': 0.374595701343873, 'epoch': 2.0})

In [26]:
infer_results = [] 

model.eval()
with torch.inference_mode():
    for ex in tqdm(test_ds_text):
        _id = ex["id"]
        text = ex["text"]

        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=4096,
        ).to("cuda")

        # temperature=0.0일 때는 do_sample=False여야 합니다.
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,      
            do_sample=False,
            # temperature=0.0, # do_sample=False일 땐 생략 가능
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        input_len = inputs["input_ids"].shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        pred = 'N/A' # 찾지 못했을 때의 기본값
        found = False
        
        # 텍스트 전체에서 뒤집어서(reversed) 검사
        for char in reversed(gen_text):
            if char in ['1', '2', '3', '4', '5']:
                pred = char
                found = True
                break
        
        # 분석을 위해 더 많은 정보를 저장합니다.
        infer_results.append({
            "id": _id,
            "prediction": pred,
            "is_found": found,
            "raw_output": gen_text 
        })

print(f"완료! 총 {len(infer_results)}개 중 {sum(1 for x in infer_results if x['is_found'])}개 추출 성공")

100%|██████████| 119/119 [02:02<00:00,  1.03s/it]

완료! 총 119개 중 119개 추출 성공


In [28]:

df_pred = pd.DataFrame(infer_results) # 여기에는 이미 'prediction' 컬럼이 있습니다.

df_analysis = pd.merge(
    df_pred, 
    valid_df[['id', 'paragraph', 'question', 'answer']], 
    on='id', 
    how='left'
)

df_analysis.rename(columns={'answer': 'ground_truth'}, inplace=True)

accuracy = (df_analysis['prediction'].astype(str) == df_analysis['ground_truth'].astype(str)).mean()

print(f"✨ 최종 검증 정확도: {accuracy:.4f}")
print(f"추출 성공률: {df_analysis['is_found'].mean():.2%}")

df_analysis.to_csv("val_results_no_RAG.csv", index=False, encoding='utf-8-sig')

✨ 최종 검증 정확도: 0.7059
추출 성공률: 100.00%


In [41]:
import torch
import gc

# 기존 객체 삭제
if 'trainer' in locals(): del trainer
if 'model' in locals(): del model

# 가비지 컬렉션 및 CUDA 캐시 비우기
gc.collect()
torch.cuda.empty_cache()

### With RAG

In [31]:
import faiss
import pandas as pd
from FlagEmbedding import BGEM3FlagModel

### 검색
model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True)
index = faiss.read_index("wikipedia_bge_m3.index")
df = pd.read_parquet("wikipedia_chunks_meta.parquet")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1165197 entries, 0 to 1165196
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   doc_id    1165197 non-null  int64 
 1   title     1165197 non-null  object
 2   text      1165197 non-null  object
 3   chunk_id  1165197 non-null  object
dtypes: int64(1), object(3)
memory usage: 35.6+ MB


In [33]:
def get_retrieved_context(row, top_k=1):
    query = f"{row['paragraph']} \n\n {row['question']}"
    
    q_vec = model.encode([query], return_dense=True)['dense_vecs']
    
    q_vec = q_vec.astype("float32")
    
    # 검색
    distances, indices = index.search(q_vec, top_k)
    
    retrieved_docs = []
    for i in indices[0]:
        if i != -1:
            retrieved_docs.append(df.iloc[i]['text'])
            
    return "\n\n".join(retrieved_docs)

In [35]:
tqdm.pandas()

valid_df['retrieved_context'] = valid_df.progress_apply(get_retrieved_context, axis=1)
train_df['retrieved_context'] = train_df.progress_apply(get_retrieved_context, axis=1)

100%|██████████| 673/673 [21:15<00:00,  1.89s/it]


In [51]:
train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

dataset = DatasetDict({
    "train": train_ds,
    "validation": valid_ds
})

In [70]:
train_ds

Dataset({
    features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len', 'retrieved_context', '__index_level_0__'],
    num_rows: 673
})

In [71]:
USER_PROMPTS = {
    4: {
        "v1": {
            "plus": USER_PROMPT_PLUS_4_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_V1,
        },
        # 추가: RAG 전용 버전
        "rag_v1": {
            "plus": USER_PROMPT_PLUS_4_RAG_V1,
            "no_plus": USER_PROMPT_NO_PLUS_4_RAG_V1,
        },
        "rag_v2": {
            "plus": USER_PROMPT_PLUS_4_RAG_V2,
            "no_plus": USER_PROMPT_NO_PLUS_4_RAG_V2,
        },
    
    },
    5: {
        "v1": {
            "plus": USER_PROMPT_PLUS_5_V1,
            "no_plus": USER_PROMPT_NO_PLUS_5_V1,
        },
        # 필요시 5지선다 RAG 템플릿도 추가
    }
}

# 정책을 rag_v1으로 변경
USER_PROMPT_POLICY = {
    4: "rag_v2",
    5: "v1",
}

In [72]:
def get_user_message(row, user_prompts, prompt_policy):
    choices_len = row["choices_len"]
    version = prompt_policy[choices_len]
    template_set = user_prompts[choices_len][version]
    
    paragraph = row['paragraph']
    question = row['question']
    choices_str = "\n".join([f"{i+1}. {c}" for i, c in enumerate(row['choices'])])
    q_plus = row.get('question_plus', None)
    
    # RAG 데이터 가져오기 (없을 경우 빈 문자열)
    r_context = row.get('retrieved_context', "")
    
    # 템플릿에 들어갈 인자 구성 (모든 템플릿 변수를 포함해도 무방)
    format_kwargs = {
        "paragraph": paragraph,
        "question": question,
        "choices": choices_str,
        "retrieved_context": r_context, # RAG 템플릿일 때 사용됨
        "question_plus": q_plus if q_plus and str(q_plus) != 'nan' else ""
    }
    
    if q_plus and str(q_plus) != 'nan':
        return template_set["plus"].format(**format_kwargs)
    else:
        return template_set["no_plus"].format(**format_kwargs)

In [73]:
def build_messages(example):
    """
    원본 example(row)로부터 학습용 chat messages를 구성한다.
    - choices_len(4/5) 및 question_plus 유무에 따라 system/user 프롬프트를 선택
    - assistant는 정답 숫자만
    - 이후 평가/추적용으로 id, label도 함께 유지
    """
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)
    asst_msg = get_assistant_message(example)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
            {"role": "assistant", "content": asst_msg},
        ],
        "label": int(example["answer"]),
    }


def build_test_messages(example):
    sys_msg = get_system_message(example, SYSTEM_PROMPTS, SYSTEM_PROMPT_POLICY)
    user_msg = get_user_message(example, USER_PROMPTS, USER_PROMPT_POLICY)

    return {
        "id": example["id"],
        "messages": [
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": user_msg},
        ]
    }

In [74]:
orig_cols = dataset["train"].column_names
dataset_msg = dataset.map(
    build_messages,
    batched=False,
    remove_columns=orig_cols,
    desc="Build messages",
)
dataset_text = dataset_msg.map(
    to_text,
    batched=False,
    remove_columns=["messages"],
    desc="Serialize to text",
)
tokenized_dataset = dataset_text.map(
    tokenize_fn,
    batched=True,
    fn_kwargs={"truncation": True, "max_length": 2048, "padding": False},
    num_proc=4, 
    remove_columns=["text"],
    load_from_cache_file=True,
    keep_in_memory=True,
    desc="Tokenizing",
)

### 검증용
test_ds_msg = valid_ds.map(
    build_test_messages,
    batched=False,
    desc="Build messages",
)
test_ds_text = test_ds_msg.map(
    to_test_text,
    batched=False,
    desc="Serialize to text",
)

Build messages:   0%|          | 0/673 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/119 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/673 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/119 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/673 [00:00<?, ? examples/s]

Tokenizing (num_proc=4):   0%|          | 0/119 [00:00<?, ? examples/s]

Build messages:   0%|          | 0/119 [00:00<?, ? examples/s]

Serialize to text:   0%|          | 0/119 [00:00<?, ? examples/s]

In [75]:
train_lens = [len(ids) for ids in tokenized_dataset["train"]["input_ids"]]
valid_lens = [len(ids) for ids in tokenized_dataset["validation"]["input_ids"]]

def print_stats(name, lens):
    print(f"[{name} Dataset Stats]")
    print(f"- 평균 토큰 길이: {np.mean(lens):.1f}")
    print(f"- 최대 토큰 길이: {max(lens)}")
    print(f"- 2048 초과 개수: {sum(1 for l in lens if l >= 2048)} / 전체 {len(lens)}")
    print(f"- 95퍼센타일 길이: {np.percentile(lens, 95):.1f}") # 상위 5% 수준의 길이
    print("-" * 30)

print_stats("Train", train_lens)
print_stats("Validation", valid_lens)

[Train Dataset Stats]
- 평균 토큰 길이: 1164.1
- 최대 토큰 길이: 1805
- 2048 초과 개수: 0 / 전체 673
- 95퍼센타일 길이: 1549.8
------------------------------
[Validation Dataset Stats]
- 평균 토큰 길이: 1130.7
- 최대 토큰 길이: 1839
- 2048 초과 개수: 0 / 전체 119
- 95퍼센타일 길이: 1575.2
------------------------------


In [76]:
test_ds_text[0]

{'id': 'generation-for-nlp-1261',
 'paragraph': '오, 수치스럽도다, 불쌍한 겨울의 왕이여! 그대는 도대체 무슨 짓을 벌인 것인가? 카이저의 왕좌를 찬탈하는 것은 무척이나 나쁜 일이 아니던가? 이제 그대는 라인강과 프라하 모두로부터 멀어져야 할지니, 무엇보다도 수치와 경멸에 의해 밤낮으로 괴로움에 떨 것이다. 그대와 온 세상이 잘 알고 있었고, 그들 역시도 잘 알고 있었다, 바로 페르디난트만이 보헤미아의 정당한 왕이라는 것을. 그러니 프리츠여, 일어서서 그대의 왕 페르디난트에게 가라, 그대의 왕에게 부디 그 죄를 사하게 해달라 은혜롭게 간청하라. ”불쌍한 겨울의 왕,” 17세기의 노래',
 'question': '다음 중 위 노래에 영감을 준 사건은?',
 'choices': ['아우스부르크의 평화', '스페인 왕위 계승 전쟁', '낭트칙령', '30년 전쟁'],
 'answer': 4,
 'question_plus': None,
 'choices_len': 4,
 'retrieved_context': "보헤미아 국왕시절의 베드르지흐(프리드리히 5세) '''프리드리히 5세'''(Friedrich V, 1596년 8월 26일 ~ 1632년 11월 29일)는 라인의 팔츠 선제후 겸 보헤미아 왕국의 국왕(재위: 1610년 ~ 1620년)이다. 보헤미아 국왕으로는 '''베드르지흐'''()로 불렸다. 그가 보헤미아 왕으로 재위한 후 정적인 제국 측은 그의 치세가 그해 겨울 안에 끝날 것이라는 의미의 '''겨울왕'''()이라는 별칭을 붙여 모욕했고, 실제 그의 치세가 짧은 기간에 그치면서 이 별명이 굳어지게 되었다. 프리드리히는 프랑스식 교육을 받았으며 1610년 부친 프리드리히 4세가 사망하자 승계했다. 프로테스탄트 국가인 보헤미아에서 귀족들이 로마 가톨릭교도인 오스트리아 대공 페르디난트 2세에 대항해 보헤미아 반란을 일으켰고 1619년 11월 4일 프라하에서 즉위식을 가져 보헤미아 왕위에 올랐다. 보헤미아 국왕으로 등극한 직후 베

In [77]:
response_template = "<|im_start|>assistant\n"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)
train_dataset = tokenized_dataset["train"].remove_columns(["id", "label"])
eval_dataset = tokenized_dataset["validation"].remove_columns(["id", "label"])

# 모델 이름 및 양자화 설정
MODEL_NAME = "Qwen/Qwen3-8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# 모델 로드 (양자화 및 장치 할당)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
)

# 모델 설정 업데이트
model.config.use_cache = False
model.gradient_checkpointing_enable()

# 토크나이저의 PAD 설정을 모델에 동기화
model.config.pad_token_id = tokenizer.pad_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id

# 확인 출력
print(f"Final Pad Token: {tokenizer.pad_token} ({tokenizer.pad_token_id})")
print(f"Use Cache: {model.config.use_cache}, Grad Ckpt: {model.is_gradient_checkpointing}")

model = prepare_model_for_kbit_training(model)

# Attention proj만
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]

# Attention + MLP까지 (성능 더 노리되 trainable 조금 증가)
# "gate_proj", "up_proj", "down_proj" -> FFN
# target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# target_modules = ["q_proj", "k_proj"]

target_modules = [
    "q_proj", "k_proj", "v_proj", "o_proj",
    "gate_proj", "up_proj", "down_proj"
]

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Final Pad Token: <|endoftext|> (151643)
Use Cache: False, Grad Ckpt: True
trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301


In [79]:
training_args = SFTConfig(
    output_dir="../../qwen-sft-results",
    
    num_train_epochs=2,
    max_seq_length=2048,
    packing=False,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,

    learning_rate=5e-5,
    fp16=True,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.01,
    
    # 전략 설정
    eval_strategy="steps",       # 주석 해제 (활성화)
    eval_steps=20,
    save_strategy="no",          # 저장 안 함 (비교용)
    load_best_model_at_end=False, # 저장 안 할 때는 False가 안전
    
    # 지표 및 로깅
    metric_for_best_model="accuracy",
    greater_is_better=True,
    logging_steps=10,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    args=training_args,
)

/data/ephemeral/pro-nlp-generationfornlp-nlp-13/.venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [80]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Step,Training Loss,Validation Loss,Accuracy,Macro F1
20,1.213500,0.155997,0.521008,0.415040
40,0.131600,0.124936,0.663866,0.527464
60,0.125600,0.125845,0.663866,0.525704
80,0.155300,0.131436,0.705882,0.563249
100,0.101600,0.121077,0.697479,0.554445
120,0.079900,0.120533,0.680672,0.539082
140,0.102400,0.118769,0.680672,0.538153
160,0.107200,0.116921,0.680672,0.538153


TrainOutput(global_step=170, training_loss=0.3773255818030413, metrics={'train_runtime': 2652.2344, 'train_samples_per_second': 0.507, 'train_steps_per_second': 0.064, 'total_flos': 8.001250633660416e+16, 'train_loss': 0.3773255818030413, 'epoch': 2.0})

In [81]:
infer_results3 = [] 

model.eval()
with torch.inference_mode():
    for ex in tqdm(test_ds_text):
        _id = ex["id"]
        text = ex["text"]

        retrieved_chunk = ex.get("retrieved_context") or ex.get("retrieved_chunk") or ex.get("context")
        
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=4096,
        ).to("cuda")

        # temperature=0.0일 때는 do_sample=False여야 합니다.
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,      
            do_sample=False,
            # temperature=0.0, # do_sample=False일 땐 생략 가능
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        input_len = inputs["input_ids"].shape[-1]
        gen_ids = outputs[0][input_len:]
        gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

        pred = 'N/A' # 찾지 못했을 때의 기본값
        found = False
        
        # 텍스트 전체에서 뒤집어서(reversed) 검사
        for char in reversed(gen_text):
            if char in ['1', '2', '3', '4', '5']:
                pred = char
                found = True
                break
        
        # 분석을 위해 더 많은 정보를 저장합니다.
        infer_results3.append({
            "id": _id,
            "prediction": pred,
            "is_found": found,
            "raw_output": gen_text,
            "retrieved_chunk": retrieved_chunk, 
        })

print(f"완료! 총 {len(infer_results3)}개 중 {sum(1 for x in infer_results3 if x['is_found'])}개 추출 성공")

  0%|          | 0/119 [00:00<?, ?it/s]

100%|██████████| 119/119 [02:37<00:00,  1.32s/it]

완료! 총 119개 중 119개 추출 성공


In [82]:
df_pred = pd.DataFrame(infer_results3) # 여기에는 이미 'prediction' 컬럼이 있습니다.

df_analysis = pd.merge(
    df_pred, 
    valid_df[['id', 'paragraph', 'question', 'answer']], 
    on='id', 
    how='left'
)

df_analysis.rename(columns={'answer': 'ground_truth'}, inplace=True)

accuracy = (df_analysis['prediction'].astype(str) == df_analysis['ground_truth'].astype(str)).mean()

print(f"✨ 최종 검증 정확도: {accuracy:.4f}")
print(f"추출 성공률: {df_analysis['is_found'].mean():.2%}")

df_analysis.to_csv("val_results_RAG2.csv", index=False, encoding='utf-8-sig')

✨ 최종 검증 정확도: 0.6807
추출 성공률: 100.00%
